In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train= False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

In [2]:
learning_rate =1e-3
batch_size = 64
epochs = 5

In [3]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [4]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [5]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309604  [   64/60000]
loss: 2.291197  [ 6464/60000]
loss: 2.272173  [12864/60000]
loss: 2.259893  [19264/60000]
loss: 2.230398  [25664/60000]
loss: 2.210742  [32064/60000]
loss: 2.225853  [38464/60000]
loss: 2.186698  [44864/60000]
loss: 2.173594  [51264/60000]
loss: 2.139097  [57664/60000]
Test Error: 
 Accuracy: 42.9%, Avg loss: 2.139707 

Epoch 2
-------------------------------
loss: 2.148560  [   64/60000]
loss: 2.142079  [ 6464/60000]
loss: 2.082522  [12864/60000]
loss: 2.101881  [19264/60000]
loss: 2.033279  [25664/60000]
loss: 1.982239  [32064/60000]
loss: 2.020083  [38464/60000]
loss: 1.933199  [44864/60000]
loss: 1.926446  [51264/60000]
loss: 1.865647  [57664/60000]
Test Error: 
 Accuracy: 56.6%, Avg loss: 1.862114 

Epoch 3
-------------------------------
loss: 1.889396  [   64/60000]
loss: 1.865584  [ 6464/60000]
loss: 1.745624  [12864/60000]
loss: 1.795562  [19264/60000]
loss: 1.662280  [25664/60000]
loss: 1.628302  [32064/600